# Creating a synthetic timeline

In [13]:
import random
from datetime import date, datetime, timedelta
from random import choice
import os
from pathlib import Path
from enum import Enum, auto
import pandas as pd
import seaborn as sns

In [2]:
##### Predefined items to pick from
locations = ["chidos", "labas", "living_room", "VU", "carl_home", "carla_home", "hospital", "club", "health_center"]
activities = ["doctor", "dentist", "hair", "museum", "swim", "walk","gym", "lunch", "breakfast", "dinner", "shop", 'cook', "clean", "wash", "visit", "read", "tv", "bridge"]
friends = ["carla", "fred", "john", "mary"]

In [7]:

class Certainty(Enum):
    UNDERSPECIFIED = 0
    POSSIBLE = 1
    PROBABLE = 2
    CERTAIN = 3

    @staticmethod
    def from_str(label):
        try:
            return Certainty[label.upper()]
        except:
            return Certainty.UNDERSPECIFIED

    @staticmethod
    def from_float(numeric):
        if numeric > .90:
            return Certainty.CERTAIN
        elif numeric >= .50:
            return Certainty.PROBABLE
        elif numeric > 0:
            return Certainty.POSSIBLE
        else:
            return Certainty.UNDERSPECIFIED

    @staticmethod
    def as_enum(value):
        # Convert to enum
        if value is None:
            return Certainty.UNDERSPECIFIED
        elif isinstance(value, Certainty):
            return value
        elif isinstance(value, int) or isinstance(value, float):
            return Certainty.from_float(value)
        elif isinstance(value, str) and value.isnumeric():
            return Certainty.from_float(float(value))
        elif isinstance(value, str):
            return Certainty.from_str(value)
        else:
            return Certainty.UNDERSPECIFIED


class Polarity(Enum):
    UNDERSPECIFIED = 0
    NEGATIVE = -1
    POSITIVE = 1

    @staticmethod
    def from_str(label):
        try:
            return Polarity[label.upper()]
        except:
            return Polarity.UNDERSPECIFIED

    @staticmethod
    def from_float(numeric):
        if numeric >= 0:
            return Polarity.POSITIVE
        elif numeric < 0:
            return Polarity.NEGATIVE

    @staticmethod
    def as_enum(value):
        # Convert to enum
        if value is None:
            return Polarity.UNDERSPECIFIED
        elif isinstance(value, Polarity):
            return value
        elif isinstance(value, int) or isinstance(value, float):
            return Polarity.from_float(value)
        elif isinstance(value, str) and value.isnumeric():
            return Polarity.from_float(float(value))
        elif isinstance(value, str):
            return Polarity.from_str(value)
        else:
            return Polarity.UNDERSPECIFIED


class Sentiment(Enum):
    UNDERSPECIFIED = 0
    NEGATIVE = -1
    POSITIVE = 1
    SOME_POS = 0.5
    NEUTRAL = 0
    SOME_NEF = -0.5
    VERY_NEG = -1.5
    VERY_POS = 1.5
    EXTREME_NEG = -2
    EXTREME_POS = 2

    @staticmethod
    def from_str(label):
        try:
            return Sentiment[label.upper()]
        except:
            return Sentiment.UNDERSPECIFIED


    @staticmethod
    def from_value(label):
        try:
            return int(Sentiment[label.upper()])
        except:
            return int(Sentiment.UNDERSPECIFIED)

    @staticmethod
    def from_float(numeric):
        if numeric > 0:
            return Sentiment.POSITIVE
        elif numeric < 0:
            return Sentiment.NEGATIVE
        elif numeric == 0:
            return Sentiment.NEUTRAL

    @staticmethod
    def as_enum(value):
        # Convert to enum
        if value is None:
            return Sentiment.UNDERSPECIFIED
        elif isinstance(value, Sentiment):
            return Sentiment(value)
        elif isinstance(value, int) or isinstance(value, float):
            return Sentiment.from_float(value)
        elif isinstance(value, str) and value.isnumeric():
            return Sentiment.from_float(float(value))
        elif isinstance(value, str):
            return Sentiment.from_str(value)
        else:
            return Sentiment.UNDERSPECIFIED

class GoNegEmotion(Enum):
    FEAR = auto()
    NERVOUSNESS = auto()
    REMORSE = auto()
    EMBARRASSMENT = auto()
    DISAPPOINTMENT = auto()
    SADNESS = auto()
    GRIEF = auto()
    DISGUST = auto()
    ANGER = auto()
    ANNOYANCE = auto()
    DISAPPROVAL = auto()
    REALIZATION = auto()
    CONFUSION = auto()

    @staticmethod
    def random():
        emotion= choice(list(GoNegEmotion))
        return emotion.name

class GoPosEmotion(Enum):
    AMUSEMENT = auto()
    EXCITEMENT = auto()
    JOY = auto()
    LOVE = auto()
    DESIRE = auto()
    OPTIMISM = auto()
    CARING = auto()
    PRIDE = auto()
    ADMIRATION = auto()
    GRATITUDE = auto()
    RELIEF = auto()
    APPROVAL = auto()
    SURPRISE = auto()
    CURIOSITY = auto()

    @staticmethod
    def random():
        emotion = choice(list(GoPosEmotion))
        return emotion.name

In [36]:

def create_an_event(human:str, event_date:datetime, activity_type):
    location = choice(locations)
    activity = choice(activities)
    friend = choice(friends)
    i = choice(range(6,20)) ### pick a random hour
    updated_time = event_date + timedelta(hours=i)
    certainty = choice(list(Certainty))
    sentiment = choice(list(Sentiment))
    polarity = choice(list(Polarity))
    emotion = 'neutral'
    if sentiment.value < -0.2:
        emotion = GoNegEmotion.random()
    elif sentiment.value > 0.2:
        emotion = GoPosEmotion.random()
    event_data = {
        "time": updated_time,
        "location": location,
        "activity_label": activity,
        "activity_type": activity_type,
        "actor1": human,
        "actor2": friend,
        "author": human,
        "certainty": certainty,
        "sentiment": sentiment,
        "emotion": emotion,
        "polarity": polarity
    }
    return event_data

def create_a_life(human: str, start: date, end: date, leap:int, nr:int, activity_type="activity"):
    life = []
    event_date = start
    while event_date<end:
        nr_events = random.randint(1, nr)
        for i in range(nr_events):
            event = create_an_event(human, event_date, activity_type=activity_type)
            life.append(event)
        ### jump in time some days random from deltas
        days = random.randint(1, leap)
        event_date = event_date+timedelta(days=days)
    return life

In [37]:
start = datetime(2024, 2, 10)
end = datetime(2024, 2, 20)
leap = 1
nr_events_per_day = 2
a_life = create_a_life(human="carl", start=start, end=end, leap=leap, nr=nr_events_per_day, activity_type="icf")

In [38]:
print(a_life)

[{'time': datetime.datetime(2024, 2, 10, 6, 0), 'location': 'chidos', 'activity_label': 'breakfast', 'activity_type': 'icf', 'actor1': 'carl', 'actor2': 'mary', 'author': 'carl', 'certainty': <Certainty.CERTAIN: 3>, 'sentiment': <Sentiment.POSITIVE: 1>, 'emotion': 'JOY', 'polarity': <Polarity.UNDERSPECIFIED: 0>}, {'time': datetime.datetime(2024, 2, 10, 16, 0), 'location': 'living_room', 'activity_label': 'breakfast', 'activity_type': 'icf', 'actor1': 'carl', 'actor2': 'carla', 'author': 'carl', 'certainty': <Certainty.PROBABLE: 2>, 'sentiment': <Sentiment.VERY_NEG: -1.5>, 'emotion': 'EMBARRASSMENT', 'polarity': <Polarity.UNDERSPECIFIED: 0>}, {'time': datetime.datetime(2024, 2, 11, 10, 0), 'location': 'hospital', 'activity_label': 'read', 'activity_type': 'icf', 'actor1': 'carl', 'actor2': 'carla', 'author': 'carl', 'certainty': <Certainty.PROBABLE: 2>, 'sentiment': <Sentiment.UNDERSPECIFIED: 0>, 'emotion': 'neutral', 'polarity': <Polarity.NEGATIVE: -1>}, {'time': datetime.datetime(2024

In [39]:
df = pd.DataFrame(a_life)
df.head()

,time,location,activity_label,activity_type,actor1,actor2,author,certainty,sentiment,emotion,polarity
0,2024-02-10 06:00:00,chidos,breakfast,icf,carl,mary,carl,Certainty.CERTAIN,Sentiment.POSITIVE,JOY,Polarity.UNDERSPECIFIED
1,2024-02-10 16:00:00,living_room,breakfast,icf,carl,carla,carl,Certainty.PROBABLE,Sentiment.VERY_NEG,EMBARRASSMENT,Polarity.UNDERSPECIFIED
2,2024-02-11 10:00:00,hospital,read,icf,carl,carla,carl,Certainty.PROBABLE,Sentiment.UNDERSPECIFIED,neutral,Polarity.NEGATIVE
3,2024-02-11 13:00:00,chidos,dinner,icf,carl,fred,carl,Certainty.PROBABLE,Sentiment.EXTREME_NEG,CONFUSION,Polarity.UNDERSPECIFIED
4,2024-02-12 07:00:00,chidos,gym,icf,carl,john,carl,Certainty.POSSIBLE,Sentiment.EXTREME_POS,DESIRE,Polarity.NEGATIVE
